In [1]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import os
import numpy as np
import shutil
tf.summary.FileWriterCache.clear()
shutil.rmtree('D:/MATLAB/OSU/OSU-automapping/Train/log/', ignore_errors=True)

sess = tf.InteractiveSession()

In [2]:
# Define functions

def conv2d(x, W, dy ,dx):
  return tf.nn.conv2d(x, W, strides=[1, dx, dy, 1], padding='SAME')

def max_pool_axb(x,a,b):
  return tf.nn.max_pool(x, ksize=[1, a, b, 1],
                        strides=[1, a, b, 1], padding='SAME')

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

# def variable_summaries(var):
#   """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
#   with tf.name_scope('summaries'):
#     mean = tf.reduce_mean(var)
#     tf.summary.scalar('mean', mean)
#     with tf.name_scope('stddev'):
#       stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
#     tf.summary.scalar('stddev', stddev)
#     tf.summary.scalar('max', tf.reduce_max(var))
#     tf.summary.scalar('min', tf.reduce_min(var))
#     tf.summary.histogram('histogram', var)

# Number of neurons , filters and layers

In [3]:
# Define the parameters of the network

#-----------ACC 0.774 *** score 81----------------
nx_conv1=16      # filter_width (frequency)
my_conv1=4   # filter_height (time)
dnx_conv1=4    # stride_width
dmy_conv1=1    # stride_height

nx_pool1=4      # pooling width
my_pool1=1      # pooling height

channels_conv1=64    # channels num

nx_conv2=2
my_conv2=16
dnx_conv2=1
dmy_conv2=8

nx_pool2=2
my_pool2=1


channels_conv2=32

size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
num_neurons_fc1=128

print(size_input_fc1)




# nx_conv1=16      # filter_width (frequency)
# my_conv1=4   # filter_height (time)
# dnx_conv1=1    # stride_width
# dmy_conv1=2    # stride_height

# nx_pool1=4      # pooling width
# my_pool1=2      # pooling height

# channels_conv1=32    # channels num

# nx_conv2=2
# my_conv2=16
# dnx_conv2=1
# dmy_conv2=8

# nx_pool2=2
# my_pool2=1


# channels_conv2=8

# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/nx_pool1/my_pool1/nx_pool2/my_pool2)
# num_neurons_fc1=64

# print(size_input_fc1)

2048


# Network Structure

In [4]:
x = tf.placeholder("float", shape=[None, 128*128])
y_ = tf.placeholder("float", shape=[None, 4])


x_image = tf.reshape(x, [-1,128,128,1])

W_conv1 = weight_variable([nx_conv1,my_conv1,1,channels_conv1])
b_conv1 = bias_variable([channels_conv1])


h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, dnx_conv1, dmy_conv1 ) + b_conv1)
h_pool1 = max_pool_axb(h_conv1, nx_pool1, my_pool1)


W_conv2 = weight_variable([nx_conv2, my_conv2, channels_conv1, channels_conv2])
b_conv2 = bias_variable([channels_conv2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, dnx_conv2, dmy_conv2 ) + b_conv2)
h_pool2 = max_pool_axb(h_conv2, nx_pool2, my_pool2)



W_fc1 = weight_variable([size_input_fc1, num_neurons_fc1])
b_fc1 = bias_variable([num_neurons_fc1])

h_pool2_flat = tf.reshape(h_pool2, [-1, size_input_fc1])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)


W_fc2 = weight_variable([num_neurons_fc1, 4])
b_fc2 = bias_variable([4])



keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)



cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


# Read test data and initialize the session

In [5]:
testData=sio.loadmat('C:/OSU/SongMat/Test/532522 SakiZ - osu!memories.mat')
inputTest=testData['input'].reshape(-1,128*128)
targetTest=testData['target'].reshape(-1,4)

NTest=targetTest.shape[0]

In [6]:
sess.run(tf.global_variables_initializer())

In [7]:
saver = tf.train.Saver()


In [8]:
step_size=100
 
dataList=os.listdir('C:/OSU/SongMat/Train/')
acc=np.zeros([1000,1])


tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('cross_entropy',cross_entropy)

merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('D:/MATLAB/OSU/OSU-automapping/Train/log/train',sess.graph)
test_writer = tf.summary.FileWriter('D:/MATLAB/OSU/OSU-automapping/Train/log/test')


n = 1
total_steps = 0

while n < np.size(dataList)-15:
    inputOsu=np.empty([0,128*128])
    targetOsu=np.empty([0,4])
    for k in range(n,n+15):
        
        data=sio.loadmat('C:/OSU/SongMat/Train/'+dataList[k])
        inputOsu=np.append(inputOsu,data['input1'].reshape(-1,128*128),axis=0)
        targetOsu=np.append(targetOsu,data['target1'].reshape(-1,4),axis=0)
               
    N=targetOsu.shape[0]
    print(N)
    
    #shuffle the order
    order = np.arange(N-step_size)
    np.random.shuffle(order)
    
    for i in range(N-step_size):
        if i%step_size == 0:
            train_accuracy = accuracy.eval(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: targetOsu[order[i:i+step_size],:], keep_prob: 1.0})
#             print("step %d, training accuracy %g" %(i, train_accuracy))
#             train_step.run(feed_dict={x: inputOsu[order[i:i+step_size],:], y_: targetOsu2[order[i:i+step_size],:], keep_prob: 0.5})
            [summary,accTrain,_]=sess.run([merged,accuracy,train_step],feed_dict={x: inputOsu[order[i:i+step_size],:], y_: targetOsu[order[i:i+step_size],:], keep_prob: 0.5})
            test_writer.add_summary(summary, i + total_steps)
#         else:
#             train_writer.add_summary(summary, i + total_steps)
                
    total_steps += i
    
    
    
    # Test accuracy
    orderAcc = np.arange(NTest)
    np.random.shuffle(orderAcc)
    accTest=accuracy.eval(feed_dict={x: inputTest[orderAcc[0:300],:], y_: targetTest[orderAcc[0:300],:], keep_prob: 1.0});
#     print("train accuracy %g"%accTrain)
    print("test accuracy %g"%accTest)
    
    print(y_conv.eval(feed_dict={x: inputTest[orderAcc[0:10],:], keep_prob: 1.0}))
    acc[n]=accTest
    print(n)
    if acc[n]>0.8:
        break
    n=n+2
        

18000
test accuracy 0.583333
[[ 0.15981978  0.06132326  0.17136712  0.60748982]
 [ 0.15079373  0.0623415   0.13543428  0.65143043]
 [ 0.04101622  0.01253377  0.03536366  0.91108632]
 [ 0.20511925  0.07980452  0.1321266   0.58294958]
 [ 0.24709551  0.12209065  0.15563384  0.47517994]
 [ 0.13744105  0.0551135   0.15856458  0.6488809 ]
 [ 0.21747279  0.10516161  0.22265232  0.45471323]
 [ 0.25246125  0.13775998  0.20255002  0.40722877]
 [ 0.2309622   0.09292988  0.18898357  0.48712438]
 [ 0.29307935  0.16463862  0.19339055  0.34889156]]
1
17019
test accuracy 0.563333
[[ 0.27540052  0.19266242  0.11782102  0.414116  ]
 [ 0.16846536  0.0642752   0.14371403  0.62354547]
 [ 0.04472848  0.0060489   0.03939522  0.90982741]
 [ 0.14834395  0.03298911  0.07149056  0.74717635]
 [ 0.3810975   0.42190894  0.14682326  0.05017033]
 [ 0.38118812  0.26363373  0.14273824  0.21243988]
 [ 0.42760658  0.26823932  0.16007988  0.14407422]
 [ 0.39126787  0.21738817  0.2197272   0.17161673]
 [ 0.31485683  0.1923

24654
test accuracy 0.676667
[[ 0.4960596   0.27394316  0.14088142  0.08911574]
 [ 0.31229064  0.04830289  0.27474764  0.36465883]
 [ 0.30548304  0.07297985  0.21269315  0.4088439 ]
 [ 0.63215667  0.20691316  0.14596803  0.0149622 ]
 [ 0.47722214  0.30816787  0.13617112  0.07843886]
 [ 0.18600434  0.05750968  0.13173987  0.62474614]
 [ 0.42096502  0.08169335  0.28073153  0.21661007]
 [ 0.0831892   0.00660923  0.028045    0.88215661]
 [ 0.58555216  0.14985761  0.19592234  0.06866792]
 [ 0.40963247  0.07163986  0.17016241  0.34856522]]
29
22131
test accuracy 0.61
[[ 0.12176897  0.00866994  0.07503379  0.79452729]
 [ 0.22424011  0.06790307  0.26821825  0.43963861]
 [ 0.3429763   0.08993907  0.22593026  0.34115437]
 [ 0.46385273  0.38222161  0.13399291  0.01993268]
 [ 0.04268776  0.00144441  0.02060008  0.93526775]
 [ 0.16560972  0.01177231  0.06022498  0.76239294]
 [ 0.13501765  0.01937293  0.05481937  0.79079008]
 [ 0.05736954  0.00881661  0.05306205  0.88075185]
 [ 0.04243447  0.0019478

In [9]:
save_path = saver.save(sess, "D:/OSU/results/save/",global_step=n)

In [10]:
print(y_conv.eval(feed_dict={x: inputOsu[200:210:1,:], keep_prob: 1.0}))
print ("test accuracy %f"%accuracy.eval(feed_dict={
  x: inputTest[orderAcc[0:300],:], y_: targetOsu[orderAcc[0:300],:], keep_prob: 1.0}))

[[  2.38590147e-02   1.08548941e-03   1.45507138e-02   9.60504770e-01]
 [  3.35697353e-01   5.82750261e-01   7.29547665e-02   8.59767664e-03]
 [  8.35176650e-03   1.00400811e-03   5.99811086e-03   9.84646142e-01]
 [  2.02232823e-01   9.91686434e-02   3.66752595e-01   3.31845939e-01]
 [  4.53046530e-01   3.70025694e-01   1.53637826e-01   2.32898965e-02]
 [  2.09164922e-03   6.22757580e-05   1.60723145e-03   9.96238828e-01]
 [  3.57324630e-01   2.48157114e-01   2.85969973e-01   1.08548328e-01]
 [  1.89202856e-02   8.65620561e-04   9.61411372e-03   9.70599949e-01]
 [  3.03301036e-01   5.34801543e-01   1.28238738e-01   3.36586870e-02]
 [  1.28005771e-02   4.18091018e-04   6.89289952e-03   9.79888380e-01]]
test accuracy 0.343333


In [11]:
# Save the output of new maps 

dataList=os.listdir('C:/OSU/SongMat/Create/')
# inputOsuNew=np.empty([0,128*128]);
for i in range(np.size(dataList)):
    if not dataList[i][0]=='y':
        data=sio.loadmat('C:/OSU/SongMat/Create/'+dataList[i])
        print(data['osuFolder'])
        
        inputOsuNew=data['input'].reshape(-1,128*128)
        outputOsuNew=data['target']
        
        y_result=np.empty([0,4])
        
        step_size=200
        cursor=0
        print ("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsuNew[-200:,:], y_: outputOsuNew[-200:,:], keep_prob: 1.0}))
        while cursor<inputOsuNew.shape[0]:
            if inputOsuNew[cursor:,:].shape[0]>step_size:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            else:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            cursor+=step_size
            
        data_save={'y':y_result}
        sio.savemat('C:/OSU/SongMat/Create/y_'+dataList[i],data_save)


[ 'D:\\Program Files (x86)\\osu!\\Songs\\450315 Sayuri - Sore wa Chiisana Hikari no Youna\\']
test accuracy 0.56
[ 'D:\\Program Files (x86)\\osu!\\Songs\\450406 Triad Primus - Trancing Pulse\\']
test accuracy 0.51
[ 'D:\\Program Files (x86)\\osu!\\Songs\\450767 Enjoykin - Lampovaya Nyasha\\']
test accuracy 0.715
[ 'D:\\Program Files (x86)\\osu!\\Songs\\451070 Pierce The Veil - Texas Is Forever\\']
test accuracy 0.635
[ 'D:\\Program Files (x86)\\osu!\\Songs\\451311 Tonight Alive - Little Lion Man\\']
test accuracy 0.79
[ 'D:\\Program Files (x86)\\osu!\\Songs\\451316 FLOW - GO!!! ~NARUTO OPENING MIX~\\']
test accuracy 0.64
[ 'D:\\Program Files (x86)\\osu!\\Songs\\451560 WALKUERE - Ichido dake no Koi nara\\']
test accuracy 0.495
[ 'D:\\Program Files (x86)\\osu!\\Songs\\451830 Nanamori-chu_Gorakubu - Acchu~ma Seishun!\\']
test accuracy 0.78
[ 'D:\\Program Files (x86)\\osu!\\Songs\\452310 Wagakki Band - Valkyrie-Ikusa Otome-\\']
test accuracy 0.57
[ 'D:\\Program Files (x86)\\osu!\\Songs\\45

In [ ]:
data_save={'y':y_result}
sio.savemat('D:/OSU/results/innocence/y_innocence.mat',data_save)

In [ ]:
checkpoint_dir='D:/OSU/results/innocence/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
saver2 = tf.train.Saver()

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver2.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess_tmp = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess_tmp.run(c))

In [ ]:
data_test=sio.loadmat('/Users/dongqihan/Desktop/FrOstNova/granat.mat')
inputOsu_test=data_test['input'].reshape(-1,128*128)
y_result_test=y_conv.eval(feed_dict={x: inputOsu_test, keep_prob: 1.0})
data={'y':y_result_test}
sio.savemat('/Users/dongqihan/Desktop/FrOstNova/y_memory2granat.mat',data)